In [33]:
### Imports
import re, time, requests, pandas as pd
from dateutil.rrule import rrule, WEEKLY
from datetime import datetime
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Kocsis Eszter
[nltk_data]     Diána\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h3>Billboard Top100 - Prepare raw dataset</h3>

In [ ]:
## Open datasets
billboard_df = pd.read_json("./data/hot100_2000_2025_raw.json")
genius_df = pd.read_json("NEW_genius_lyrics_raw.json")

In [35]:
# Correcting the date format after opening the json file
billboard_df["chart_date"] = pd.to_datetime(billboard_df["chart_date"], unit="ms", errors="coerce")
billboard_df["chart_date"] = billboard_df["chart_date"].dt.strftime("%Y-%m-%d")
billboard_df

,chart_date,rank,title,artist,last_week,peak,weeks_on_chart
0,2000-01-08,1,Smooth,SantanaFeaturingRob Thomas,1.0,1,24
1,2000-01-08,2,Back At One,Brian McKnight,2.0,2,20
2,2000-01-08,3,I Wanna Love You Forever,Jessica Simpson,3.0,3,13
3,2000-01-08,4,My Love Is Your Love,Whitney Houston,4.0,4,19
4,2000-01-08,5,Hot Boyz,"Missy ""Misdemeanor"" Elliott FeaturingNas,Eve&Q...",7.0,5,7
...,...,...,...,...,...,...,...
133595,2025-08-09,96,Madeline,Zach BryanFeaturing Gabriella Rose,62.0,62,2
133596,2025-08-09,97,Mommanem,"Tyler, the Creator",54.0,54,2
133597,2025-08-09,98,River Washed Hair,Zach Bryan,90.0,53,4
133598,2025-08-09,99,Don't Tap That Glass / Tweakin',"Tyler, the Creator",56.0,56,2


In [36]:
# Clean the raw dataset
def clean_artist_name(name):
    # space around &
    name = re.sub(r'\s*&\s*', ' & ', name)
    # space around "and"
    name = re.sub(r'\s*(and)\s*', r' and ', name, flags=re.IGNORECASE)
    # space around "Featuring"
    name = re.sub(r'\s*(featuring)\s*', r' Featuring ', name, flags=re.IGNORECASE)
    # space around "Feat"
    name = re.sub(r'\s*(feat\.)\s*', r' Feat. ', name, flags=re.IGNORECASE)
    # space around "Presents"
    name = re.sub(r'\s*(presents)\s*', r' Presents ', name, flags=re.IGNORECASE)
    # space around "With"
    name = re.sub(r'\s*(with)\s*', r' With ', name, flags=re.IGNORECASE)
    # space around ","
    name = re.sub(r'\s*,\s*', ', ', name)
    # remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()

    return name

billboard_df["artist"] = billboard_df["artist"].apply(clean_artist_name)

## later added adjustment based on the first results - Ariana Grande
billboard_df["artist"] = billboard_df["artist"].str.replace(r"Gr\s*and\s*e", "Grande", regex=True)

billboard_df

,chart_date,rank,title,artist,last_week,peak,weeks_on_chart
0,2000-01-08,1,Smooth,Santana Featuring Rob Thomas,1.0,1,24
1,2000-01-08,2,Back At One,Brian McKnight,2.0,2,20
2,2000-01-08,3,I Wanna Love You Forever,Jessica Simpson,3.0,3,13
3,2000-01-08,4,My Love Is Your Love,Whitney Houston,4.0,4,19
4,2000-01-08,5,Hot Boyz,"Missy ""Misdemeanor"" Elliott Featuring Nas, Eve...",7.0,5,7
...,...,...,...,...,...,...,...
133595,2025-08-09,96,Madeline,Zach Bryan Featuring Gabriella Rose,62.0,62,2
133596,2025-08-09,97,Mommanem,"Tyler, the Creator",54.0,54,2
133597,2025-08-09,98,River Washed Hair,Zach Bryan,90.0,53,4
133598,2025-08-09,99,Don't Tap That Glass / Tweakin',"Tyler, the Creator",56.0,56,2


In [37]:
# Filter dataset by timerange - 2018-2024
billboard_df = billboard_df.copy()
billboard_df["chart_date"] = pd.to_datetime(billboard_df["chart_date"]).dt.date

mask = (billboard_df["chart_date"] >= pd.to_datetime("2018-01-01").date()) & \
       (billboard_df["chart_date"] <= pd.to_datetime("2024-12-31").date())
billboard_2018_2024 = billboard_df[mask].copy()

billboard_2018_2024

,chart_date,rank,title,artist,last_week,peak,weeks_on_chart
93900,2018-01-06,1,Perfect,Ed Sheeran,1.0,1,18
93901,2018-01-06,2,Rockstar,Post Malone Featuring 21 Savage,2.0,1,15
93902,2018-01-06,3,Havana,Camila Cabello Featuring Young Thug,3.0,2,20
93903,2018-01-06,4,No Limit,G-Eazy Featuring A$AP Rocky & Cardi B,5.0,4,16
93904,2018-01-06,5,Thunder,Imagine Dragons,6.0,4,35
...,...,...,...,...,...,...,...
130395,2024-12-28,96,Gloria,Kendrick Lamar & SZA,76.0,27,4
130396,2024-12-28,97,Diet Pepsi,Addison Rae,NaN,54,13
130397,2024-12-28,98,Disease,Lady Gaga,NaN,27,7
130398,2024-12-28,99,Shake Dat Ass (Twerk Song),BossMan Dlow,NaN,54,13


<h2>Preparation for cleaning the lyrics</h2>

In [38]:
# Remove duplicate entries from billboard
unique_songs_df = billboard_2018_2024[["title", "artist"]].drop_duplicates().reset_index(drop=True)
unique_songs_df

,title,artist
0,Perfect,Ed Sheeran
1,Rockstar,Post Malone Featuring 21 Savage
2,Havana,Camila Cabello Featuring Young Thug
3,No Limit,G-Eazy Featuring A$AP Rocky & Cardi B
4,Thunder,Imagine Dragons
...,...,...
4415,El Club,Bad Bunny
4416,Rosones,Tito Double P
4417,Toxic Till The End,ROSE
4418,White Christmas,V & Bing Crosby


In [39]:
# Filter genius dataset - just the relevant songs are going to stay
genius_filtered = genius_df.merge(unique_songs_df[["artist", "title"]], on=["artist", "title"], how="inner")
genius_filtered

,title,artist,genius_id,language,primary_tag,tags,genius_url,release_date,album,lyrics
0,All I Want For Christmas Is You,Mariah Carey,204233.0,en,R&B,"[R&B, Pop, USA, In English, Soul Pop, Soul, Po...",https://genius.com/Mariah-carey-all-i-want-for...,1994-10-29,Greatest Hits,185 Contributors\nTranslations\nEspañol\nTürkç...
1,Bye Bye Bye,*NSYNC,115478.0,en,Pop,"[Pop, Alternative Dance, Soundtrack, USA, In E...",https://genius.com/Nsync-bye-bye-bye-lyrics,2000-01-11,The Essential *NSYNC,94 Contributors\nTranslations\nDeutsch\nEspaño...
2,Still D.R.E.,Dr. Dre Featuring Snoop Dogg,7701008.0,en,Non-Music,"[Non-Music, R&B, Rap, In English, USA, America...",https://genius.com/Dr-dre-snoop-dogg-eminem-ma...,2022-02-13,Super Bowl Halftime Shows,43 Contributors\nSuper Bowl LVI Halftime Show ...
3,Party Up (Up In Here),DMX,4402.0,en,Rap,"[Rap, Hip-Hop, New York Rap, In English, USA, ...",https://genius.com/Dmx-party-up-up-in-here-lyrics,1999-12-21,Verzuz: DMX x Snoop Dogg,138 Contributors\nParty Up (Up in Here) Lyrics...
4,The Next Episode,Dr. Dre Featuring Snoop Dogg,7701008.0,en,Non-Music,"[Non-Music, R&B, Rap, In English, USA, America...",https://genius.com/Dr-dre-snoop-dogg-eminem-ma...,2022-02-13,Super Bowl Halftime Shows,43 Contributors\nSuper Bowl LVI Halftime Show ...
...,...,...,...,...,...,...,...,...,...,...
4415,El Club,Bad Bunny,11180508.0,es,Pop,"[Pop, Deep House, Latin Pop, House, Latin Urba...",https://genius.com/Bad-bunny-el-club-lyrics,2024-12-05,DeBÍ TiRAR MáS FOToS,28 Contributors\nTranslations\nEnglish\nDeutsc...
4416,Rosones,Tito Double P,11098618.0,es,Pop,"[Pop, Cover, En Español, México, Latin Music, ...",https://genius.com/Tito-double-p-rosones-lyrics,2024-12-05,None,8 Contributors\nTranslations\nEnglish\nROSONES...
4417,Toxic Till The End,ROSE,11112065.0,en,Pop,"[Pop, NZ Pop, K-Pop (케이팝), In English, Genius ...",https://genius.com/Rose-toxic-till-the-end-lyrics,2024-12-06,rosie (vampirehollie edition),62 Contributors\nTranslations\nالعربية\nItalia...
4418,White Christmas,V & Bing Crosby,11199532.0,es,Pop,"[Pop, Traducción Al Español]",https://genius.com/Genius-traducciones-al-espa...,2024-12-06,None,1 Contributor\nTranslations\nEnglish\nV & Bing...


In [40]:
# Remove the non-english songs
genius_filtered_en = genius_filtered[genius_filtered["language"] == "en"].copy()
genius_filtered_en

,title,artist,genius_id,language,primary_tag,tags,genius_url,release_date,album,lyrics
0,All I Want For Christmas Is You,Mariah Carey,204233.0,en,R&B,"[R&B, Pop, USA, In English, Soul Pop, Soul, Po...",https://genius.com/Mariah-carey-all-i-want-for...,1994-10-29,Greatest Hits,185 Contributors\nTranslations\nEspañol\nTürkç...
1,Bye Bye Bye,*NSYNC,115478.0,en,Pop,"[Pop, Alternative Dance, Soundtrack, USA, In E...",https://genius.com/Nsync-bye-bye-bye-lyrics,2000-01-11,The Essential *NSYNC,94 Contributors\nTranslations\nDeutsch\nEspaño...
2,Still D.R.E.,Dr. Dre Featuring Snoop Dogg,7701008.0,en,Non-Music,"[Non-Music, R&B, Rap, In English, USA, America...",https://genius.com/Dr-dre-snoop-dogg-eminem-ma...,2022-02-13,Super Bowl Halftime Shows,43 Contributors\nSuper Bowl LVI Halftime Show ...
3,Party Up (Up In Here),DMX,4402.0,en,Rap,"[Rap, Hip-Hop, New York Rap, In English, USA, ...",https://genius.com/Dmx-party-up-up-in-here-lyrics,1999-12-21,Verzuz: DMX x Snoop Dogg,138 Contributors\nParty Up (Up in Here) Lyrics...
4,The Next Episode,Dr. Dre Featuring Snoop Dogg,7701008.0,en,Non-Music,"[Non-Music, R&B, Rap, In English, USA, America...",https://genius.com/Dr-dre-snoop-dogg-eminem-ma...,2022-02-13,Super Bowl Halftime Shows,43 Contributors\nSuper Bowl LVI Halftime Show ...
...,...,...,...,...,...,...,...,...,...,...
4412,KTM Drip,Juice WRLD,6766139.0,en,Rap,"[Rap, Midwest Rap, Alternative, Alternative Ra...",https://genius.com/Juice-wrld-ktm-drip-lyrics,2024-11-29,The Party Never Ends (Deluxe),149 Contributors\nTranslations\nУкраїнська\nРу...
4413,Cuffed,Juice WRLD,6539310.0,en,Rap,"[Rap, USA, Leak, Hip-Hop, Dark Trap, In Englis...",https://genius.com/Juice-wrld-cuffed-lyrics,2024-11-29,The Party Never Ends (Deluxe),"78 Contributors\nCuffed Lyrics\nOn “Cuffed,“ J..."
4414,Winter Ahead,V With Park Hyo Shin,11141687.0,en,Pop,"[Pop, K-Ballad, In English, Jazz, Duet, South ...",https://genius.com/V-and-park-hyo-shin-winter-...,2024-11-29,Winter Ahead (Silent Carol Ver.),20 Contributors\nTranslations\nEspañol\n한국어 (K...
4417,Toxic Till The End,ROSE,11112065.0,en,Pop,"[Pop, NZ Pop, K-Pop (케이팝), In English, Genius ...",https://genius.com/Rose-toxic-till-the-end-lyrics,2024-12-06,rosie (vampirehollie edition),62 Contributors\nTranslations\nالعربية\nItalia...


In [41]:
## Checking the unique primary_tags (aka genre)
# List of unique tags
unique_tags = genius_filtered_en["primary_tag"].dropna().unique()
print(unique_tags)

# Count of tags
tag_counts = genius_filtered_en["primary_tag"].value_counts(dropna=True)
print(tag_counts)

['R&B' 'Pop' 'Non-Music' 'Rap' 'Rock' 'Electronic' 'Country'
 '#FlipTheSwitchChallenge' 'rage' 'Chicago drill' 'pop' 'drill' 'Funk'
 'EDM' 'Hip-Hop' 'trap' 'hyphy']
primary_tag
Rap                        1218
Pop                         662
Country                     467
Non-Music                   466
R&B                         423
Electronic                  224
Rock                        190
Chicago drill                 2
#FlipTheSwitchChallenge       1
rage                          1
pop                           1
drill                         1
Funk                          1
EDM                           1
Hip-Hop                       1
trap                          1
hyphy                         1
Name: count, dtype: int64


In [42]:
# Remove primary_tags that are "Non-Music"
genius_filtered_en = genius_filtered_en[genius_filtered_en["primary_tag"] != "Non-Music"].copy()

# Normalize the rest of the tags - lowercase
genius_filtered_en["primary_tag"] = genius_filtered_en["primary_tag"].astype(str).str.lower()
print(genius_filtered_en["primary_tag"].unique())

['r&b' 'pop' 'rap' 'rock' 'electronic' 'country' '#fliptheswitchchallenge'
 'rage' 'chicago drill' 'drill' 'funk' 'edm' 'hip-hop' 'trap' 'hyphy'
 'none']


In [43]:
# Primary_tag "none" - 1 case, Eminem -> rap
genius_filtered_en["primary_tag"] = genius_filtered_en["primary_tag"].replace("none", "rap")

<h2>Cleaning the lyrics</h2>

In [44]:
# Clean lyrics w regex function
def clean_lyrics(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return text

    # Remove texts until the first found - Intro/Verse/Chorus/Bridge/Pre-Chorus/Hook/Refrain/Outro
    section_pat = r"\[(?:.*?(?:Intro|Verse|Chorus|Bridge|Pre[- ]?Chorus|Hook|Refrain|Outro).*?)\]"
    m = re.search(section_pat, text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        text = text[m.start():]
    else:
        # Without [] blocks, start with removing the "X Contributors ... Read More" blocks
        text = re.sub(r"(?is)^\s*\d+\s+Contributors.*?(Read\s*More\s*)?\n+", "", text, count=1)
        text = re.sub(r"(?is)^\s*.*?lyrics\s*\n+", "", text, count=1)

    # Remove all of the [ ... ] and ( ... ) blocks (eg. [Verse 1], [Chorus], ...etc.)
    text = re.sub(r"\[.*?\]", "", text, flags=re.S)
    text = re.sub(r"\(.*?\)", "", text, flags=re.S)
    text = re.sub(r"\[.*?Intro.*?\]", "", text, flags=re.IGNORECASE)

    # Remove empty rowes/whitespaces
    text = re.sub(r"[ \t]+\n", "\n", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = text.strip()

    return text

# Run function - clean lyrics
genius_filtered_en.loc[:, "lyrics_clean"] = genius_filtered_en["lyrics"].apply(clean_lyrics)


# Remove stopwords
stop_words = set(stopwords.words("english"))
# extra words
extra_filters = {"oh", "ooh", "yeah", "na", "la", "ha", "woo", "du", "lol", "uh", "hey", "ah"}
stop_words = stop_words.union(extra_filters)

# Remove stopwords function
def remove_stopwords(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return text

    # words into lowercase
    words = re.findall(r"\b\w+\b", text.lower())

    # filter stopwords
    filtered = [w for w in words if w not in stop_words]

    return " ".join(filtered)

# Run function - remove stopwords
genius_filtered_en.loc[:, "lyrics_clean"] = genius_filtered_en["lyrics_clean"].apply(remove_stopwords)

In [45]:
# Just in case - remove rows with empty lyrics (just 1 case)
genius_filtered_en["lyrics_clean"] = genius_filtered_en["lyrics_clean"].replace(r"^\s*$", pd.NA, regex=True)
genius_filtered_en = genius_filtered_en.dropna(subset=["lyrics_clean"])

genius_filtered_en

,title,artist,genius_id,language,primary_tag,tags,genius_url,release_date,album,lyrics,lyrics_clean
0,All I Want For Christmas Is You,Mariah Carey,204233.0,en,r&b,"[R&B, Pop, USA, In English, Soul Pop, Soul, Po...",https://genius.com/Mariah-carey-all-i-want-for...,1994-10-29,Greatest Hits,185 Contributors\nTranslations\nEspañol\nTürkç...,want lot christmas one thing need care present...
1,Bye Bye Bye,*NSYNC,115478.0,en,pop,"[Pop, Alternative Dance, Soundtrack, USA, In E...",https://genius.com/Nsync-bye-bye-bye-lyrics,2000-01-11,The Essential *NSYNC,94 Contributors\nTranslations\nDeutsch\nEspaño...,bye bye bye doin tonight probably gonna start ...
3,Party Up (Up In Here),DMX,4402.0,en,rap,"[Rap, Hip-Hop, New York Rap, In English, USA, ...",https://genius.com/Dmx-party-up-up-in-here-lyrics,1999-12-21,Verzuz: DMX x Snoop Dogg,138 Contributors\nParty Up (Up in Here) Lyrics...,gon make lose mind gon make go gon make act fo...
5,Lose Yourself,Eminem,207.0,en,pop,"[Pop, Rap, In English, USA, Pop Rap, Detroit R...",https://genius.com/Eminem-lose-yourself-lyrics,2002-10-28,SHADYXV,826 Contributors\nTranslations\nDeutsch\nTürkç...,look one shot one opportunity seize everything...
6,X Gon' Give It To Ya,DMX,3200.0,en,rap,"[Rap, Hip-Hop, New York Rap, In English, USA, ...",https://genius.com/Dmx-x-gon-give-it-to-ya-lyrics,2003-02-18,Verzuz: DMX x Snoop Dogg,161 Contributors\nTranslations\nDeutsch\nX Gon...,arf arf get twisted rap shit mine motherfucker...
...,...,...,...,...,...,...,...,...,...,...,...
4412,KTM Drip,Juice WRLD,6766139.0,en,rap,"[Rap, Midwest Rap, Alternative, Alternative Ra...",https://genius.com/Juice-wrld-ktm-drip-lyrics,2024-11-29,The Party Never Ends (Deluxe),149 Contributors\nTranslations\nУкраїнська\nРу...,perc get let fuck hatin outside club drugs kno...
4413,Cuffed,Juice WRLD,6539310.0,en,rap,"[Rap, USA, Leak, Hip-Hop, Dark Trap, In Englis...",https://genius.com/Juice-wrld-cuffed-lyrics,2024-11-29,The Party Never Ends (Deluxe),"78 Contributors\nCuffed Lyrics\nOn “Cuffed,“ J...",hold hold hold hold gon get wild bitch yo ayy ...
4414,Winter Ahead,V With Park Hyo Shin,11141687.0,en,pop,"[Pop, K-Ballad, In English, Jazz, Duet, South ...",https://genius.com/V-and-park-hyo-shin-winter-...,2024-11-29,Winter Ahead (Silent Carol Ver.),20 Contributors\nTranslations\nEspañol\n한국어 (K...,lie fire safe storms high winter ahead cold we...
4417,Toxic Till The End,ROSE,11112065.0,en,pop,"[Pop, NZ Pop, K-Pop (케이팝), In English, Genius ...",https://genius.com/Rose-toxic-till-the-end-lyrics,2024-12-06,rosie (vampirehollie edition),62 Contributors\nTranslations\nالعربية\nItalia...,call us toxic start pretend dark met friends e...


<h2>Get date of PEAK from Billboard</h2>

In [46]:
## Get the dates of the peaks of the songs
# Logic: rank == peak
peak_songs = billboard_2018_2024[billboard_2018_2024["rank"] == billboard_2018_2024["peak"]].copy()

# Sort entries to get the lowest value
peaks_sorted = peak_songs.sort_values(by=["title", "artist", "peak", "chart_date"], ascending=[True,   True,   True,  True])

peak_dates = (peak_songs.sort_values(["title", "artist", "chart_date"]).drop_duplicates(subset=["title", "artist"], keep="first"))

# Keep only the needed columns
peak_dates = (peaks_sorted.drop_duplicates(subset=["title", "artist"], keep="first").rename(columns={"chart_date": "peak_date"})[["title", "artist", "peak", "peak_date"]])
peak_dates

,title,artist,peak,peak_date
121026,'98 Braves,Morgan Wallen,27,2023-03-18
116017,'Til You Can't,Cody Johnson,18,2022-04-02
109438,'Tis The Damn Season,Taylor Swift,39,2020-12-26
125249,(There's No Place Like) Home For The Holidays ...,Perry Como With Mitchell Ayers and His Orchestra,50,2024-01-06
109521,(There's No Place Like) Home For The Holidays ...,Perry Como With Mitchell Ayers and His Orchestra,22,2021-01-02
...,...,...,...,...
111619,punchin'.the.clock,J. Cole,20,2021-05-29
126922,thanK you aIMee,Taylor Swift,23,2024-05-04
111624,the.climb.back,J. Cole,25,2021-05-29
112175,transparentsoul,Willow Featuring Travis Barker,76,2021-07-03


In [47]:
# Add the peaks to the corresponding songs
genius_with_peak = genius_filtered_en.merge(
    peak_dates[["artist", "title", "peak", "peak_date"]],
    on=["artist", "title"],
    how="left"
)

genius_with_peak

# Save dataset
# genius_with_peak.to_json("genius_cleaned_merged.json")

,title,artist,genius_id,language,primary_tag,tags,genius_url,release_date,album,lyrics,lyrics_clean,peak,peak_date
0,All I Want For Christmas Is You,Mariah Carey,204233.0,en,r&b,"[R&B, Pop, USA, In English, Soul Pop, Soul, Po...",https://genius.com/Mariah-carey-all-i-want-for...,1994-10-29,Greatest Hits,185 Contributors\nTranslations\nEspañol\nTürkç...,want lot christmas one thing need care present...,1.0,2019-12-21
1,Bye Bye Bye,*NSYNC,115478.0,en,pop,"[Pop, Alternative Dance, Soundtrack, USA, In E...",https://genius.com/Nsync-bye-bye-bye-lyrics,2000-01-11,The Essential *NSYNC,94 Contributors\nTranslations\nDeutsch\nEspaño...,bye bye bye doin tonight probably gonna start ...,NaN,NaN
2,Party Up (Up In Here),DMX,4402.0,en,rap,"[Rap, Hip-Hop, New York Rap, In English, USA, ...",https://genius.com/Dmx-party-up-up-in-here-lyrics,1999-12-21,Verzuz: DMX x Snoop Dogg,138 Contributors\nParty Up (Up in Here) Lyrics...,gon make lose mind gon make go gon make act fo...,NaN,NaN
3,Lose Yourself,Eminem,207.0,en,pop,"[Pop, Rap, In English, USA, Pop Rap, Detroit R...",https://genius.com/Eminem-lose-yourself-lyrics,2002-10-28,SHADYXV,826 Contributors\nTranslations\nDeutsch\nTürkç...,look one shot one opportunity seize everything...,NaN,NaN
4,X Gon' Give It To Ya,DMX,3200.0,en,rap,"[Rap, Hip-Hop, New York Rap, In English, USA, ...",https://genius.com/Dmx-x-gon-give-it-to-ya-lyrics,2003-02-18,Verzuz: DMX x Snoop Dogg,161 Contributors\nTranslations\nDeutsch\nX Gon...,arf arf get twisted rap shit mine motherfucker...,46.0,2021-04-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,KTM Drip,Juice WRLD,6766139.0,en,rap,"[Rap, Midwest Rap, Alternative, Alternative Ra...",https://genius.com/Juice-wrld-ktm-drip-lyrics,2024-11-29,The Party Never Ends (Deluxe),149 Contributors\nTranslations\nУкраїнська\nРу...,perc get let fuck hatin outside club drugs kno...,91.0,2024-12-14
3191,Cuffed,Juice WRLD,6539310.0,en,rap,"[Rap, USA, Leak, Hip-Hop, Dark Trap, In Englis...",https://genius.com/Juice-wrld-cuffed-lyrics,2024-11-29,The Party Never Ends (Deluxe),"78 Contributors\nCuffed Lyrics\nOn “Cuffed,“ J...",hold hold hold hold gon get wild bitch yo ayy ...,95.0,2024-12-14
3192,Winter Ahead,V With Park Hyo Shin,11141687.0,en,pop,"[Pop, K-Ballad, In English, Jazz, Duet, South ...",https://genius.com/V-and-park-hyo-shin-winter-...,2024-11-29,Winter Ahead (Silent Carol Ver.),20 Contributors\nTranslations\nEspañol\n한국어 (K...,lie fire safe storms high winter ahead cold we...,99.0,2024-12-14
3193,Toxic Till The End,ROSE,11112065.0,en,pop,"[Pop, NZ Pop, K-Pop (케이팝), In English, Genius ...",https://genius.com/Rose-toxic-till-the-end-lyrics,2024-12-06,rosie (vampirehollie edition),62 Contributors\nTranslations\nالعربية\nItalia...,call us toxic start pretend dark met friends e...,90.0,2024-12-21
